In [ ]:
import os

os.environ["WIKIDATA_CACHE_FILE"] = "../.wikidata-cache.json"

In [ ]:
# https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html

from shifter_pandas.bp import UNITS_ENERGY, BPDatasource

shifter_ds = BPDatasource("../data/bp-stats-review-2021-all-data.xlsx")

In [ ]:
for type_ in shifter_ds.metadata():
    if not type_['supported']:
        print(type_['type'])

In [ ]:
for type_ in shifter_ds.metadata():
    if type_['supported']:
        print(f"{type_['type']} [{type_['unit']['iso']}]")
    
# TODO?
# Elec Gen by fuel

In [ ]:
shifter_ds.oil_units_conversion

In [ ]:
shifter_ds.oil_products_units_conversion

In [ ]:
shifter_ds.gaz_units_conversion

In [ ]:
r = {}
for e in shifter_ds.metadata():
    if type_['supported']:
        r[e["unit"]["original"]] = f'[{e["unit"]["iso"]}]{e["unit"]["iso_postfix"]} => {e["unit"]["iso_factor"]}'
r

In [ ]:
{e["unit"]["iso"] for e in shifter_ds.metadata() if type_['supported']}

In [ ]:
shifter_ds.to_iso_unit

In [ ]:
df = shifter_ds.datasource_non_fossil_electricity_to_primary_energy_factor(1995)
df

In [ ]:
import plotly.express as px

fig = px.line(df, x="Year", y="Factor")
fig.show()

In [ ]:
df = shifter_ds.datasource(
    regions_filter=["Total World"],
    types_filter=[
'Nuclear Generation',
'Nuclear Consumption',
    ],
)
for col in ["TypeUnit", "Region"]:
    if col in df:
        del df[col]
df

In [ ]:
import plotly.express as px

fig = px.line(df, x="Year", y="Value", color="Type")
fig.show()

In [ ]:
df_gene = df.query('Type == "Nuclear Generation"')
df_gene.rename(columns={"Value": "NuclearGeneration"}, inplace=True)
del df_gene['Type']
del df_gene['Unit']
df_cons = df.query('Type == "Nuclear Consumption"')
df_cons.rename(columns={"Value": "NuclearConsumption"}, inplace=True)
del df_cons['Type']
del df_cons['Unit']

In [ ]:
import pandas as pd

df_merged = pd.merge(
    df_gene,
    df_cons,
    how="inner",
    on=["Year"],
)
df_merged

In [ ]:
df_final = df_merged.assign(NuclearRatio=df_merged.NuclearGeneration / df_merged.NuclearConsumption)
#df_final = df_merged.assign(NuclearRatio=df_merged.NuclearConsumption / df_merged.NuclearGeneration)
df_final

In [ ]:
import plotly.express as px

fig = px.line(df_final, x="Year", y="NuclearRatio")
fig.show()

In [ ]:
df = shifter_ds.datasource(
    regions_filter=["Total World"],
    types_filter=[
'Oil Consumption - Barrels',
'Oil Consumption - Tonnes',
'Oil Consumption - EJ'
    ],
)
for col in ["TypeUnit", "Region"]:
    if col in df:
        del df[col]
df

In [ ]:
df_vol = df.query('Type == "Oil Consumption - Barrels"')
df_vol.rename(columns={"Value": "OilVolume"}, inplace=True)
del df_vol['Type']
del df_vol['Unit']
df_mass = df.query('Type == "Oil Consumption - Tonnes"')
df_mass.rename(columns={"Value": "OilMass"}, inplace=True)
del df_mass['Type']
del df_mass['Unit']
df_energy = df.query('Type == "Oil Consumption - EJ"')
df_energy.rename(columns={"Value": "OilEnergy"}, inplace=True)
del df_energy['Type']
del df_energy['Unit']
df_energy

In [ ]:
import pandas as pd

df_merged = pd.merge(
    df_vol,
    df_mass,
    how="inner",
    on=["Year"],
)
df_merged = pd.merge(
    df_merged,
    df_energy,
    how="inner",
    on=["Year"],
)
df_merged

In [ ]:
df_final = df_merged.assign(OilEnergyVolume=df_merged.OilEnergy / df_merged.OilVolume)
df_final = df_final.assign(OilEnergyMass=df_merged.OilEnergy / df_merged.OilMass)
df_final = df_final.assign(OilMassVolume=df_merged.OilMass / df_merged.OilVolume)
df_final

In [ ]:
import plotly.express as px

fig = px.line(df_final, x="Year", y="OilEnergyVolume")
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df_final, x="Year", y="OilEnergyMass")
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df_final, x="Year", y="OilMassVolume")
fig.show()